In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re

# TOP-250 фильмов по версии ~~КиноПоиска~~ Mail.ru
Обнаружила, что КиноПоиск не любит, когда с него выгружают информацию, поэтому я нашла аналогичный топ в разделе КИНО на Mail.ru. Причем прошлые страницы c Кинопоиска удалось раскопать, потому что они находятся на старой части сайта, в разделе личного кабинета. Страницы фильмов и топов имеют другой интерфейс и, как оказалось, другую политику конфиденциальности.

In [2]:
# загрузим файл.

with open("D:/MovieDB/CreatingTables/top250/0.html", 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'html.parser')

## HTML, содержащий нужную информацию выглядит как-то так
Часть с названием, страной, годом и жанрами:

    <div class="p-itemevent-small__info">
        <span class="text text_block text_fixed text_light_large">1. 
            <a class="link link_inline link-holder link-holder_itemevent link-holder_itemevent_small" href="/cinema/movies/624334_zelenaja_milja/">
                <span class="link__text">
                    Зеленая миля
                </span>
            </a>
        </span>
        <span class="text text_block text_fixed text_light_small color_gray margin_bottom_5">
            The Green Mile
        </span>
        <div class="margin_top_5">
            <a class=" p_link_black" href="/cinema/all/usa/">
                США
            </a>, 
            <a class=" p_link_black" href="/cinema/all/1999/">
                1999
            </a>, 
            <a class=" p_link_black" href="/cinema/all/drama/">драма</a>, 
            <a class=" p_link_black" href="/cinema/all/detektiv/">
                детектив
            </a>, 
            <a class=" p_link_black" href="/cinema/all/kriminal/">
                криминал</a>, 
            <a class=" p_link_black" 
               href="/cinema/all/fentezi/">
                фэнтези
            </a>
        </div>
    </div>
   
 ---
 И с оценками пользователей Mail и IMDB:
 
    <div class="p-rate-flag p-rate-flag_medium margin_right_5">
        <div class="p-rate-flag__inner">
            <svg class="icon icon_media-ui icon_indent_top-n1 icon_fill_current icon_svg icon_media-ui margin_right_5" height="18" width="18">
                <use class="icon__use" xlink:href="/-/35cf95fd/bem/web/web.bundles/common/common.svg#icon_actions_rating">
                </use>
            </svg>
            <span class="p-rate-flag__text">
                8.9
            </span>
        </div>
        <div class="p-rate-flag__imdb-inner p-rate-flag__imdb-inner_left">
            <span class="p-rate-flag__imdb-title">
                IMDb
            </span>
            <span class="p-rate-flag__imdb-text">
                8.6
            </span>
        </div>
    </div>

In [3]:
# создадим списки с нужными блоками кода.

infos = soup.findAll('div', {'class': 'p-itemevent-small__info'})
ratings_raw = soup.findAll('div', {'class': 'p-rate-flag p-rate-flag_medium margin_right_5'})

In [4]:
# проверим, сколько элементов получилось в каждом списке.
# почему-то список с основной информацией содержит в 2 раза больше значений, чем нужно.

len(infos), len(ratings_raw)

(500, 250)

In [5]:
# оказывается, в этом списке значения идут через одно.

for i in range(10):
    print(infos[i].find('span', {'class': 'link__text'}))

<span class="link__text">Зеленая миля</span>
None
<span class="link__text">Побег из Шоушенка</span>
None
<span class="link__text">Иван Васильевич меняет профессию</span>
None
<span class="link__text">Бриллиантовая рука</span>
None
<span class="link__text">Операция «Ы» и другие приключения Шурика</span>
None


In [6]:
# не будем разбираться, почему так, и просто удалим лишние значения.

clean_infos = []
for i in range(0, len(infos), 2):
    clean_infos.append(infos[i])

In [7]:
# соберем все данные в списки.
# обратим внимание на то, что стран, как и жанров, может быть несколько.

ru_names = []
en_names = []
countries = []
years = []
genres = []
ratings = []
imdb_ratings = []

for info, rating in zip(clean_infos, ratings_raw):
    ru_name = info.find('span', {'class': 'link__text'}).text
    is_en_name = info.find('span', {'class': 'text text_block text_fixed text_light_small color_gray margin_bottom_5'})
    if is_en_name:
        en_name = is_en_name.text
    else:
        en_name = ''
    
    others = info.find('div', {'class': 'margin_top_5'}).findAll('a', {'class': 'p_link_black'})
    genre = []
    country = []
    for other in others:
        try:
            year = int(other.text)
        except:
            regex = '[А-Я]'
            if re.match(regex, str(other.text)[0]):
                country.append(other.text)
            else:
                genre.append(other.text)            
    
    rate = float(rating.find('span', {'class': 'p-rate-flag__text'}).text)
    imdb_rate = float(rating.find('span', {'class': 'p-rate-flag__imdb-text'}).text)
    
    ru_names.append(ru_name)
    en_names.append(en_name)
    genres.append(genre)
    years.append(year)
    countries.append(country)
    ratings.append(rate)
    imdb_ratings.append(imdb_rate)

In [8]:
# создадим функцию, которая будет собирать списки в датафрейм.
 
def df_templater():
    return {'ru_name': ru_names,
           'en_name': en_names,
           'country': countries,
           'year': years,
           'genre': genres,
           'rating': ratings,
           'imdb_rating': imdb_ratings}

In [9]:
# соберем датафрейм.

top250 = pd.DataFrame(df_templater())
top250.head()

,ru_name,en_name,country,year,genre,rating,imdb_rating
0,Зеленая миля,The Green Mile,[США],1999,"[драма, детектив, криминал, фэнтези]",8.9,8.6
1,Побег из Шоушенка,The Shawshank Redemption,[США],1994,"[драма, криминал]",8.8,9.3
2,Иван Васильевич меняет профессию,,[СССР],1973,"[комедия, приключения, семейный, фантастика]",8.7,8.2
3,Бриллиантовая рука,,[СССР],1968,"[комедия, криминал, приключения, семейный]",8.6,8.3
4,Операция «Ы» и другие приключения Шурика,,[СССР],1965,[комедия],8.6,8.5


In [11]:
# сохраним файл.

with open('D:/MovieDB/CreatingTables/top250/top250.csv', 'w', encoding='utf-8', newline='') as f:
    f.write(top250.to_csv())